# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [1]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv duckduckgo_search==5.3.1b1

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

#### 🏗️ **Activity #1:**

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ **Question #1:**

How does the model determine which tool to use?

<span style="color:indigo">Each tool has a description that tells the model when it should be used. In the case of third-party off the shelf tools, you may have to look at the source code to find this. For example, the ArxivQueryRun tool has a description as follows:

<span style="color:indigo">description: str = (
        "A wrapper around Arxiv.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on arxiv.org. "
        "Input should be a search query."
    )

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
compiled_graph = uncompiled_graph.compile()

#### ❓ **Question #2:**

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

<span style="color:indigo">In the graph we compiled in the notebook above, there is no limit to the number of cycles and we could in theory encounter an infinite loop. According to LangGraph's documentation (https://langchain-ai.github.io/langgraph/how-tos/recursion-limit/#define-our-graph) we can set a recursion_limit parameter when invoking the compiled graph. This will set an upper limit on the number of loops our graph is allowed to execute.

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TZH3q4kLHyUxj7oSCtw3QrNP', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 156, 'total_tokens': 181}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-093ce2bd-8e30-460e-98fa-0cf3cabe39aa-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_TZH3q4kLHyUxj7oSCtw3QrNP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 25, 'total_tokens': 181})]



Receiving update from node: 'action'
[ToolMessage(content='Adam Lowry was named captain of the Winnipeg Jets on Tuesday. ... Hughes named Canucks captain, replaces Horvat Sep 11, 2023. 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [20]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search every author to find out their latest Tweet using DuckDuckGo. You MUST call DuckDuckGo before finishing.")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yIAzyQ2M2ZK7lQyuKosUpN00', 'function': {'arguments': '{"query":"QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 180, 'total_tokens': 196}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1075f14d-58ce-4bd8-8a1e-1d1b041a9e7b-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_yIAzyQ2M2ZK7lQyuKosUpN00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 16, 'total_tokens': 196})]



Receiving update from node: 'action'
Tool Used: arxiv
[ToolMessage(content='Published: 2023-05-23\nTitle: QLoRA: Efficient Finetuning of Quantized LLMs\nAuthors: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, Luke Zettlemoyer\nSummary: We present QLoRA, an efficient finetuning appr

#### 🏗️ **Activity #2:**

Please write out the steps the agent took to arrive at the correct answer.

<span style="color:indigo">The original system was finishing after just calling Arxiv, so I modified the system prompt above to reinforce that it must use DuckDuckGo before finishing. Here is the final flow:

1. <span style="color:indigo">Our state object was populated with our request
2. <span style="color:indigo">The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. <span style="color:indigo">The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. <span style="color:indigo">The action node called the arXiv search tool, added the response from the OpenAI function calling endpoint to the state object, and passed it along the edge back to the agent node
5. <span style="color:indigo">The agent node evaluated the state, added a response to the state object and passed it along the conditional edge
6. <span style="color:indigo">The conditional edge received the state object, found the "tool_calls" `additional_kwarg` and passed the state object to to the action node again
7. <span style="color:indigo">The action node called the DuckDuckGo API, added the response to the state object, and passed it along the edge back to the agent node
8. <span style="color:indigo">The agent node added a response to the state object and passed it along the conditional edge
9. <span style="color:indigo">There conditional object found no "tool_calls", so it passed the state object to END

<span style="color:indigo">At a higher level of abstraction, the resulting flow is:
* <span style="color:indigo">agent calls the arxiv action
* <span style="color:indigo">agent evaluates the output from arxiv and calls the duckduckgo action
* <span style="color:indigo">agent evaluates the output from duckduckgo and decides it is done

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [21]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [28]:
agent_chain.invoke({"question" : "What is the most recent stable release of LangChain? Include your sources"})

'The most recent stable release of LangChain is version 0.1.0. This release includes significant improvements in stability, focus, and developer experience. \n\nSources:\n- [LangChain v0.1.0 Release Announcement](https://www.langchain.com/blog/langchain-v0-1-0)'

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ **Activity #3:**

Please create a dataset in the above format with at least 5 questions.

In [29]:
questions = [
    "What year was Lucy Maud Montgomery born?",
    "What was L. M. Montgomery's most famous book?"
    "What was the profession of L. M. Montgomery's husband?",
    "Where were most of L. M. Montgomery's novels and stories set?",
    "How did L. M. Montgomery die?"
]

answers = [
    {"must_mention" : ["1874"]},
    {"must_mention" : ["Anne","Green","Gables"]},
    {"must_mention" : ["Presbyterian", "minister"]},
    {"must_mention" : ["Prince", "Edward", "Island"]},
    {"must_mention" : ["suicide", "overdose"]}
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [30]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ **Question #3:**

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

<span style="color:indigo">They're associated only by the position in each list, so that element 1 of questions corresponds to element 1 of answers and so forth. This is a brittle approach that can easily become messed up if something is inadverently moved or added in one list without a corresponding change in the other. It is also brittle if the test set is long. It would be better to associate each question-answer pair in a data structure such as a dict, tuple, specially written class, etc.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [31]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ **Question #4:**

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

<span style="color:indigo">Ways to improve it:
* <span style="color:indigo">Change it from binary to float, for example counting the number of occurrences of the required word and returning a fraction between 0 and 1, or using "any" == True: 0.5, all == True: 1, else 0</span>
* <span style="color:indigo">Make the metric more robust by first converting to lower case and removing punctuation</span>

<span style="color:indigo">Gaps:
In general, this is not robust to synonyms or underlying meanings.
* <span style="color:indigo">It ignores equivalent phrases that are based on word order, for example "United States" would score 0 if the must_mention value was "United States of America"</span>
* <span style="color:indigo">It ignores equivalencies based on acronyms, for example "USA" scores 0 if must_mention is "United States of America"</span>
* <span style="color:indigo">It ignores equivalent meanings and synonyms</span>
</span>

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it!

In [33]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [ ]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - f6033df6' at:
https://smith.langchain.com/o/340cd80b-3296-5752-9a9e-58582118073a/datasets/044e1430-58d6-4c1b-8e0d-48b2468c02f2/compare?selectedSessions=7c54b424-0bf0-45bc-9d5e-51db9222988c

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - 21ea3856 at:
https://smith.langchain.com/o/340cd80b-3296-5752-9a9e-58582118073a/datasets/044e1430-58d6-4c1b-8e0d-48b2468c02f2
[------------------------------------------------->] 6/6
 Experiment Results:
       feedback.must_mention error  execution_time                                run_id
count                      6     0            6.00                                     6
unique                     2     0             NaN                                     6
top                     True   NaN             NaN  15a79e0c-17eb-4c96-b22a-8aee7327d436
freq                       4   NaN             NaN                                     1
mean       

{'project_name': 'RAG Pipeline - Evaluation - f6033df6',
 'results': {'b5ae289c-27f0-4422-90c5-8b599adc4305': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='must_mention', score=True, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('32acacc7-e614-4c27-923d-68189e3fdb81'), target_run_id=None)],
   'execution_time': 2.860901,
   'run_id': '15a79e0c-17eb-4c96-b22a-8aee7327d436',
   'output': 'QLoRA (Quantized Low-Rank Adaptation) uses **paged optimizers** to manage memory spikes during the finetuning process. This approach is part of the innovations introduced by QLoRA to efficiently finetune large language models while reducing memory usage.',
   'reference': {'must_mention': ['paged', 'optimizer']}},
  '34b468a7-7087-4082-b779-3e853f793a98': {'input': {'question': 'What data type was created in the QLoRA paper?'},
   'feedback': [EvaluationResult(key='must_mention', score=True, valu

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [34]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ **Activity #5:**

Please write markdown for the following cells to explain what each is doing.

<span style="color:indigo">Initialize the state graph, and add our agent and action nodes as before.</span>

In [36]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

<span style="color:indigo">Tell the graph to call the agent first when the application first enters the loop</span>

In [37]:
graph_with_helpfulness_check.set_entry_point("agent")

<span style="color:indigo">Set up a new function to decide what to do after the agent runs. This can be used as a conditional edge place of the "should_continue" that we used before. After defining it, add it as a conditional edge below.


In [38]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ **Activity #4:**

Please write what is happening in our `tool_call_or_helpful` function!

* <span style="color:indigo">If the last message (from the agent) indicates that we must call a tool, call the action node</span>
* <span style="color:indigo">Otherwise, if we've already accumulated 10 messages, exit the loop. This avoids infinite looping.
* <span style="color:indigo">Otherwise, we have a potential final response; construct and run a chain to evaluate the helpfulness metric </span>
* <span style="color:indigo">If the response is not helpful, call the agent again, trying to generate a more helpful response. If it is helpful, exit the loop</span>
</span>

In [39]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

<span style="color:indigo">Above, we add our "tool_call_or_helpful" function as a conditional edge to evaluate each time our agent generates a message. Below, We add the edge from the action back to the agent, so that the agent is called each time the action node completes.

In [40]:
graph_with_helpfulness_check.add_edge("action", "agent")

<span style="color:indigo">Finally, compile the new graph into a runnable so that we can  call it with invoke() and other standard Langchain operations; name this compiled graph "agent_with_helpfulness_check"

In [41]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

<span style="color:indigo">Test out the newly compiled graph with an input message that has three questions. Invoke the graph with "astream" to asynchronously stream tokens back to the user as they are generated. For each node that is invoked, print out the messages it generated so that we can follow the flow. From the generated output we see that we:
1. <span style="color:indigo"> Called the agent, which decides to do DuckDuckGo search on LoRA machine learning, Tim Dettmers, and attention in machine learning
2. <span style="color:indigo"> Called the action node, which uses the teool and returns the three search results. It would be better if this also returned the links / references for better traceability and transparency.
3. <span style="color:indigo">Called the agent node to generate a final response. 
Although not outputted below, the final response must have been evaluated as helpful, because the loop finishes without trying again.

In [45]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}
from pprint import pprint
async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        pprint(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_q1W1wjL8jfJPaLrxbHKXYJkn', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_5nUVS0gVKQf3tNoN9PYq7fhU', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_Ga1NqOf1O8ZjvqGmU4UrneZ3', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 171, 'total_tokens': 247}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c476f1f3-f120-4de1-be34-0f4b6b684888-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'LoRA machine learning'}, 'id': 'call_q1W1wjL8jfJPaLrxbHKXYJkn', 'type': 'tool_call'}, 

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [46]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [47]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is a concept primarily associated with the field of artificial intelligence, particularly in the context of natural language processing (NLP) and large language models like GPT-3. It involves the design and crafting of prompts (input text) to elicit desired responses from AI models. The goal is to optimize the input to get the most accurate, relevant, or useful output from the model.

### Key Aspects of Prompt Engineering:
1. **Crafting Effective Prompts**: Designing prompts that are clear, specific, and structured in a way that the AI can understand and respond to appropriately.
2. **Iterative Testing**: Continuously refining prompts based on the responses received to improve the quality and relevance of the output.
3. **Understanding Model Behavior**: Gaining insights into how the model interprets different types of prompts and using this knowledge to guide prompt design.

### Emergence of Prompt Engineering:
Prompt engineering became more prominent with the advent